In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow

In [ ]:
def traffic_light(green_time, yellow_time, red_time):
    """
    Simulates a traffic light with time.sleep for accurate timing.

    Returns:
    - True if a violation is detected during the first red light period, False otherwise.
    """
    while True:
        print("Green light")
        time.sleep(green_time)
        print("Yellow light")
        time.sleep(yellow_time)
        print("Red light")
        time.sleep(red_time)
        print("-" * 10)

        # Check if it's the first red light period
        if red_time == 5:
            if vehicle_detected_during_red_light():
                return True
            else:
                print("No violations detected during the first red light period. Continuing simulation.")
                continue

        return False

def vehicle_detected_during_red_light():
    return True

def is_vehicle_in_detection_zone(vehicle_center, detection_zone):
    """
    Checks if the detected vehicle's center falls within the detection zone.

    Parameters:
    - vehicle_center: Tuple (x, y) representing the center of the detected vehicle.
    - detection_zone: List of tuples [(x1, y1), (x2, y2)] representing the detection zone coordinates.

    Returns:
    - True if the vehicle's center is within the detection zone, False otherwise.
    """
    x, y = vehicle_center
    x1, y1 = detection_zone[0]
    x2, y2 = detection_zone[1]
    return x1 <= x <= x2 and y1 <= y <= y2

# Set timings (in seconds)
green_time = 6
yellow_time = 3
red_time = 5

if traffic_light(green_time, yellow_time, red_time):
    print("Vehicle detected during the first red light period. Stopping simulation.")
else:
    print("No violations detected during the first red light period. Continuing simulation.")

# Create background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# Load video
cap = cv2.VideoCapture("/content/drive/MyDrive/traffic_violation/1.mp4")


detection_zone = [(900, 300), (1950, 500)]

# Flag to track if a violation is detected
violation_detected = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)

    # Apply morphology operations to remove noise
    kernel = np.ones((5,5),np.uint8)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Process each contour
    for contour in contours:
        # Get bounding box
        x, y, w, h = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 1000:
            continue

        vehicle_center = (x + w // 2, y + h // 2)

        # Check if the center of the vehicle is within the detection zone
        if is_vehicle_in_detection_zone(vehicle_center, detection_zone):
            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Draw detection zone rectangle on the frame
    cv2.rectangle(frame, detection_zone[0], detection_zone[1], (255, 0, 0), 2)

    # Display the result
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.